# Scenario
The data-set contains aggregate individual statistics for <b>67 NBA seasons</b> from basic box-score attributes such as points, assists, rebounds etc., to more advanced money-ball like features such as Value Over Replacement. 

## Pre-Processing
In this task not all data will be used, <b>only data in 2017</b>. So it is necessary to do filtering at the beginning. Besides that there are some players who make team transfers in the NBA transfer market so that there is duplication of player data. Therefore you can use the df.drop_duplicates() syntax to solve this to produce the same output as the trainer. Delete columns that have as many missing values as the entire row of data. Then you can do additional preprocessing if needed or you can immediately process the data. 

## Goals
1.	Who is the youngest and oldest player in the NBA in 2017 for each team (Tm) ?
2.	Which player has the most minutes played (MP) in each position (Pos)? 
3.	Which team has the highest average total rebound percentage (TRB%), assist percentage (AST%), steal percentage (STL%), and block percentage (BLK%)? 
4.	Who is the best player in your opinion based on his record stats? note: you can refer to variables point (PTS), assists, rebounds, or anything else. A combination of several variables would be nice. 
5.	Which team has the best average stat record of their players? Note: you can refer to points, assists, rebounds, or anything else. A combination of several variables would be nice

# Hands-On
# Import Module

In [1]:
import pandas as pd
import requests
import io
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

# Import Data

In [2]:
url = "https://raw.githubusercontent.com/Syukrondzeko/Fellowship/71eaf279b1cda677b1f6ec025409163c1a629035/Seasons_Stats.csv"
download = requests.get(url).content
data = pd.read_csv(io.StringIO(download.decode('utf-8')))

In [3]:
#Summary of the dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24691 entries, 0 to 24690
Data columns (total 53 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  24691 non-null  int64  
 1   Year        24624 non-null  float64
 2   Player      24624 non-null  object 
 3   Pos         24624 non-null  object 
 4   Age         24616 non-null  float64
 5   Tm          24624 non-null  object 
 6   G           24624 non-null  float64
 7   GS          18233 non-null  float64
 8   MP          24138 non-null  float64
 9   PER         24101 non-null  float64
 10  TS%         24538 non-null  float64
 11  3PAr        18839 non-null  float64
 12  FTr         24525 non-null  float64
 13  ORB%        20792 non-null  float64
 14  DRB%        20792 non-null  float64
 15  TRB%        21571 non-null  float64
 16  AST%        22555 non-null  float64
 17  STL%        20792 non-null  float64
 18  BLK%        20792 non-null  float64
 19  TOV%        19582 non-nul

# Select Data -> 2017

In [4]:
df = data[data['Year']==2017].reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595 entries, 0 to 594
Data columns (total 53 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  595 non-null    int64  
 1   Year        595 non-null    float64
 2   Player      595 non-null    object 
 3   Pos         595 non-null    object 
 4   Age         595 non-null    float64
 5   Tm          595 non-null    object 
 6   G           595 non-null    float64
 7   GS          595 non-null    float64
 8   MP          595 non-null    float64
 9   PER         595 non-null    float64
 10  TS%         593 non-null    float64
 11  3PAr        593 non-null    float64
 12  FTr         593 non-null    float64
 13  ORB%        595 non-null    float64
 14  DRB%        595 non-null    float64
 15  TRB%        595 non-null    float64
 16  AST%        595 non-null    float64
 17  STL%        595 non-null    float64
 18  BLK%        595 non-null    float64
 19  TOV%        593 non-null    f

## Check Missing Values

In [5]:
df.isnull().sum()

Unnamed: 0      0
Year            0
Player          0
Pos             0
Age             0
Tm              0
G               0
GS              0
MP              0
PER             0
TS%             2
3PAr            2
FTr             2
ORB%            0
DRB%            0
TRB%            0
AST%            0
STL%            0
BLK%            0
TOV%            2
USG%            0
blanl         595
OWS             0
DWS             0
WS              0
WS/48           0
blank2        595
OBPM            0
DBPM            0
BPM             0
VORP            0
FG              0
FGA             0
FG%             2
3P              0
3PA             0
3P%            46
2P              0
2PA             0
2P%             5
eFG%            2
FT              0
FTA             0
FT%            24
ORB             0
DRB             0
TRB             0
AST             0
STL             0
BLK             0
TOV             0
PF              0
PTS             0
dtype: int64

## Handle Missing Values

### 1. Drop null columns

In [6]:
df = df.drop(['blanl','blank2','Unnamed: 0'], axis=1)

Previously I used the following syntax to handle all nulls based on their formula:<br>
df['TS%'] = df['TS%'].fillna((df['PTS'])/(2*(df['TSA'])) 

But, it is inoperative. When I checked the column data that being used, surprisingly all of them were 0. 
Thus, I concluded that if the result is 0 then the column will be filled with null.

So, I filled all null values with 0 manually CMIIW

### 2. TS%

In [7]:
#TS% = PTS/(2*TSA)--> TSA not available
df[['PTS']][df['TS%'].isnull()]

,PTS
60,0.0
248,0.0


In [8]:
df['TS%'] = df['TS%'].fillna(0)

### 3. 3PAr

In [9]:
#3PAr = 3PA/FGA
df[['3PA','FGA']][df['3PAr'].isnull()]

,3PA,FGA
60,0.0,0.0
248,0.0,0.0


In [10]:
df['3PAr'] = df['3PAr'].fillna(0)

### 4. FTr

In [11]:
#FTR (Free Throw Rate) = FTA/FGA
df[['FTA','FGA']][df['FTr'].isnull()]

,FTA,FGA
60,0.0,0.0
248,0.0,0.0


In [12]:
df['FTr'] = df['FTr'].fillna(0)

### 5. TOV%

In [13]:
#TOV%=100*TOV/(FGA+0.44*FTA+TOV)
df[['TOV','FGA']][df['TOV%'].isnull()]

,TOV,FGA
60,0.0,0.0
248,0.0,0.0


In [14]:
df['TOV%'] = df['TOV%'].fillna(0)

### 6. FG%

In [15]:
#FG% = FG/FGA
df[['FG','FGA']][df['FG%'].isnull()]

,FG,FGA
60,0.0,0.0
248,0.0,0.0


In [16]:
df['FG%'] = df['FG%'].fillna(0)

### 7. 3P%

In [17]:
#3P% =  3P/3PA
df[['3P','3PA']][df['3P%'].isnull()]

,3P,3PA
7,0.0,0.0
21,0.0,0.0
24,0.0,0.0
39,0.0,0.0
51,0.0,0.0
60,0.0,0.0
86,0.0,0.0
95,0.0,0.0
98,0.0,0.0
123,0.0,0.0


In [18]:
df['3P%'] = df['3P%'].fillna(0)

### 8. 2P%

In [19]:
#2P%=2P/PA
df[['2P','2PA']][df['2P%'].isnull()]

,2P,2PA
60,0.0,0.0
248,0.0,0.0
253,0.0,0.0
351,0.0,0.0
527,0.0,0.0


In [20]:
df['2P%'] = df['2P%'].fillna(0)

### 9. eFG%

In [21]:
#eFG% = (FG + 0.5 * 3P)/FGA
df[['FG','FGA']][df['eFG%'].isnull()]

,FG,FGA
60,0.0,0.0
248,0.0,0.0


In [22]:
df['eFG%'] = df['eFG%'].fillna(0)

### 10. FT%

In [23]:
#FT%=FT/FTA
df[['FT','FTA']][df['FT%'].isnull()]

,FT,FTA
48,0.0,0.0
60,0.0,0.0
70,0.0,0.0
71,0.0,0.0
72,0.0,0.0
80,0.0,0.0
92,0.0,0.0
158,0.0,0.0
176,0.0,0.0
212,0.0,0.0


In [24]:
df['FT%'] = df['FT%'].fillna(0)

In [25]:
df.isnull().sum()

Year      0
Player    0
Pos       0
Age       0
Tm        0
G         0
GS        0
MP        0
PER       0
TS%       0
3PAr      0
FTr       0
ORB%      0
DRB%      0
TRB%      0
AST%      0
STL%      0
BLK%      0
TOV%      0
USG%      0
OWS       0
DWS       0
WS        0
WS/48     0
OBPM      0
DBPM      0
BPM       0
VORP      0
FG        0
FGA       0
FG%       0
3P        0
3PA       0
3P%       0
2P        0
2PA       0
2P%       0
eFG%      0
FT        0
FTA       0
FT%       0
ORB       0
DRB       0
TRB       0
AST       0
STL       0
BLK       0
TOV       0
PF        0
PTS       0
dtype: int64

In [26]:
df_new = df.copy()

## Drop Duplicate Values

In [27]:
#Check duplicate values
df['Player'].value_counts()

Omri Casspi         4
Ersan Ilyasova      4
Lance Stephenson    4
Hollis Thompson     3
Andrew Bogut        3
                   ..
Ron Baker           1
Jarell Martin       1
Manu Ginobili       1
Anthony Bennett     1
Solomon Hill        1
Name: Player, Length: 486, dtype: int64

In [28]:
#Drop duplicate values
df_dup = df.drop_duplicates(['Player'], keep='last')
df_dup['Player'].value_counts()

Jon Leuer          1
T.J. McConnell     1
Serge Ibaka        1
Skal Labissiere    1
Reggie Jackson     1
                  ..
Corey Brewer       1
Wayne Selden       1
Anthony Bennett    1
Jordan Farmar      1
Solomon Hill       1
Name: Player, Length: 486, dtype: int64

In [29]:
df1 = df_dup.copy()

# Questions

## Task 1
Who is the <b>youngest and oldest player</b> in the NBA in 2017 for each team (Tm) ?

In [30]:
df_young = df1[['Player','Tm','Age']].groupby(['Tm','Player']).min().sort_values(by='Tm').reset_index()
df_old = df1[['Player','Tm','Age']].groupby(['Tm']).max().sort_values(by='Tm').reset_index()

In [31]:
df_young

,Tm,Player,Age
0,ATL,DeAndre' Bembry,22.0
1,ATL,Tim Hardaway,24.0
2,ATL,Thabo Sefolosha,32.0
3,ATL,Taurean Waller-Prince,22.0
4,ATL,Ryan Kelly,25.0
...,...,...,...
481,WAS,Bradley Beal,23.0
482,WAS,Bojan Bogdanovic,27.0
483,WAS,Tomas Satoransky,25.0
484,WAS,Jason Smith,30.0


In [32]:
df_old

,Tm,Player,Age
0,ATL,Tim Hardaway,36.0
1,BOS,Tyler Zeller,31.0
2,BRK,Trevor Booker,36.0
3,CHI,Robin Lopez,35.0
4,CHO,Treveon Graham,31.0
5,CLE,Tristan Thompson,38.0
6,DAL,Yogi Ferrell,38.0
7,DEN,Wilson Chandler,36.0
8,DET,Tobias Harris,34.0
9,GSW,Zaza Pachulia,36.0


## Task 2
Which player has <b>the most minutes played (MP) in each position (Pos)</b>? 

In [33]:
df_mp = df1[['Player','Pos','MP']].groupby(['Pos']).max().sort_values(by='MP', ascending=False).reset_index()
df_mp

,Pos,Player,MP
0,SF,Wilson Chandler,3048.0
1,C,Zaza Pachulia,3030.0
2,PG,Yogi Ferrell,2947.0
3,PF,Zach Randolph,2803.0
4,SG,Zach LaVine,2796.0


## Task 3
Which team has the <b>highest average</b> total rebound percentage (TRB%), assist percentage (AST%), steal percentage (STL%), and block percentage (BLK%) ? 

In [34]:
df_highestaverage = df1[['Tm','TRB%','AST%','STL%','BLK%']].groupby(['Tm']).mean().reset_index()

In [35]:
#Highest average total rebound percentage (TRB%)
df_highestaverage[['Tm','TRB%']].sort_values(by='TRB%', ascending=False).head(1)

,Tm,TRB%
29,WAS,12.735294


In [36]:
#Highest average total rebound percentage (AST%)
df_highestaverage[['Tm','AST%']].sort_values(by='AST%', ascending=False).head(1)

,Tm,AST%
7,DEN,15.723529


In [37]:
#Highest average total rebound percentage (STL%)
df_highestaverage[['Tm','STL%']].sort_values(by='STL%', ascending=False).head(1)

,Tm,STL%
17,MIN,2.413333


In [38]:
#Highest average total rebound percentage (BLK%)
df_highestaverage[['Tm','BLK%']].sort_values(by='BLK%', ascending=False).head(1)

,Tm,BLK%
16,MIL,2.741176


## Task 4
Who is the <b>best player</b> in your opinion based on his record stats? note: you can refer to variables point (PTS), assists, rebounds, or anything else. A combination of several variables would be nice. 

In [39]:
df1['Overall'] = df1.iloc[:,5:].mean(axis=1)

In [40]:
df1[['Overall','Player']].sort_values(by='Overall', ascending=False).reset_index(drop=True).head(5)

,Overall,Player
0,360.796867,Russell Westbrook
1,328.918111,James Harden
2,302.964289,Karl-Anthony Towns
3,298.494067,Anthony Davis
4,281.170644,LeBron James


## Task 5

Which <b>team has the best average stat record</b> of their players? Note: you can refer to points, assists, rebounds, or anything else. A combination of several variables would be nice

In [41]:
df1[['Tm','Overall']].groupby(by='Tm').sum().sort_values(by='Overall',ascending=False).reset_index().head(10)

,Tm,Overall
0,GSW,1575.927800
1,WAS,1514.843933
2,LAC,1507.355156
3,BOS,1507.239800
4,SAS,1506.784778
5,MIN,1499.052933
6,IND,1497.508533
7,DET,1492.004267
8,DEN,1483.263667
9,HOU,1472.435978
